In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import dshai.utils as ds

import gspread
from oauth2client.service_account import ServiceAccountCredentials as SAC



In [2]:
# ds.hello()
ds.hello()

Hello World!


In [3]:
!pwd

/home/jedi/Desktop/dev_ds/ms_data_engineer_assessment/solution


In [4]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
creds = SAC.from_json_keyfile_name(
            'gspread-270619-05d12e0e0262.json', SCOPES)

gsp = gspread.authorize(creds)

gsp

In [5]:
base_url = 'https://docs.google.com/spreadsheets/d'
sheet_id = '1ZtGa3ykeYOK_WvE7JHoGX_zvFE4drfaHVTqtKWZ69L8'




In [6]:
gsp.open_by_url(f'{base_url}/{sheet_id}')

<Spreadsheet 'jedi_hogwarts_admissions' id:1ZtGa3ykeYOK_WvE7JHoGX_zvFE4drfaHVTqtKWZ69L8>

## Loading the data from google sheets

This fetches the raw data from google spread sheets. The API call takes some time so we run it once. We can run it again if data is updated

In [7]:
from flask_apps.services.resources.gspread_service import GspreadService
from flask_apps.services.resources.abstract_resource import AbstractSpreadSheetResource

from flask_apps.services.resources.admissions.ApplicationsRes import AllApplicationsResource
from flask_apps.services.resources.admissions.ClassRes import ClassResource
from flask_apps.services.resources.admissions.StudentsClassRes import StudentsClassResource

from flask_apps.services.resources.enrollments.ModulesRes import ModulesResource 
from flask_apps.services.resources.enrollments.BaseClassByModuleRes import BaseClassByModuleRes

from flask_apps.services.resources.outcomes.OutcomesRes import OutcomesResource

In [8]:
all_applications = AllApplicationsResource()
all_applications.fetch()
all_applications.process()

/home/jedi/anaconda3/envs/r_env/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
all_applications.parsed_data['all_applications'].shape

(6496, 8)

In [10]:
# class_ = ClassResource()
# class_.fetch()
# class_.process()

In [11]:
# student_class = StudentsClassResource()
# student_class.fetch()
# student_class.process()

In [12]:
# modules = ModulesResource()
# modules.fetch()
# modules.process()

In [13]:
# class_byMod = BaseClassByModuleRes()
# class_byMod.fetch()
# class_byMod.process()

In [14]:
# outcomes = OutcomesResource()
# outcomes.fetch()
# outcomes.process()


In [15]:
# type(outcomes.parsed_data)
import pickle

# with open('pickles/all_applications.p', "wb") as f:
#     pickle.dump(all_applications.parsed_data, f)
    
# with open('pickles/classes.p', "wb") as f:
#     pickle.dump(class_.parsed_data, f)

# with open('pickles/studentbyclass.p', "wb") as f:
#     pickle.dump(student_class.parsed_data, f)
    
# with open('pickles/modules.p', "wb") as f:
#     pickle.dump(modules.parsed_data, f)
    
# with open('pickles/classbyMod.p', "wb") as f:
#     pickle.dump(class_byMod.parsed_data, f)
    
# with open('pickles/outcomes.p', "wb") as f:
#     pickle.dump(outcomes.parsed_data, f)
    

In [16]:
with open('pickles/all_applications.p', "rb") as f:
    all_applications = pickle.load(f)
    
with open('pickles/classes.p', "rb") as f:
    classes = pickle.load(f)

with open('pickles/studentbyclass.p', "rb") as f:
    studentbyclass = pickle.load(f)
    
with open('pickles/modules.p', "rb") as f:
    modules = pickle.load(f)
    
with open('pickles/classbyMod.p', "rb") as f:
    classbyMod = pickle.load(f)
    
with open('pickles/outcomes.p', "rb") as f:
    outcomes = pickle.load(f)

In [17]:
all_applications['all_applications'].shape
# classes
# studentbyclass
# modules
# classbyMod
# outcomes

(6496, 8)

In [18]:
all_apps = all_applications['all_applications'].copy()
all_apps[['first_name', 'last_name']] = all_apps['name'].str.split(pat=' ', expand=True)[[0, 1]]

In [19]:
all_apps.shape

(6496, 8)

In [20]:
all_apps.dtypes

0
name          object
email         object
gender        object
phone         object
class         object
date          object
first_name    object
last_name     object
dtype: object

In [21]:
clean_resources = [classes, all_applications, studentbyclass]
# # for k, v in raw_admissions.items():
# #         print(f"{k:<20}:{v.shape[0]:>10}, {v.shape[1]:}")

for i in clean_resources:
    for k, v in i.items():
        print(f"{k:<20}:{v.shape[0]:>10}, {v.shape[1]:}")

classes             :        12, 4
all_applications    :      6496, 8
HC_1_students       :       400, 3
HC_2_students       :       400, 3
HC_3_students       :       400, 3
HC_4_students       :       400, 3
HC_5_students       :       400, 3
HC_6_students       :       400, 3
HC_7_students       :       400, 3
HC_8_students       :       400, 3
HC_9_students       :       400, 3
HC_10_students      :       400, 3
HC_11_students      :       400, 3
HC_12_students      :       400, 3


In [22]:
resources = ["ClassResource", "AllApplicationsResource", "StudentsClassResource"]

for i in range(len(resources)):
    print(resources[i])

ClassResource
AllApplicationsResource
StudentsClassResource


In [23]:
from flask_apps.models.AbstractModel import AbstractModel
from flask_apps.models.Classes import Classes
from flask_apps.models.Classes import ClassesSql

from flask_apps.services.db.db_handler import PostgresDBService
import psycopg2 

In [24]:
classes_ = classes['classes'].copy()
classes_[['start_date', 'end_date']] = classes_['code'].str.rsplit(pat='_', n=2, expand=True)[[1, 2]]


cls_sql = ClassesSql()

cls_indx0 = classes['classes'].iloc[0]
cls_indx1 = classes['classes'].iloc[1]

clss0 = Classes(cls_indx0['name'], cls_indx0['code'],cls_indx0['start_date'],cls_indx0['end_date'])
clss1 = Classes(cls_indx1['name'], cls_indx1['code'],cls_indx1['start_date'],cls_indx1['end_date'])

clss0.__dict__

{'id': None,
 'name': 'HC_1',
 'code': 'HC_1_2009-02-03_2009-07-21',
 'start_date': '2009-02-03',
 'end_date': '2009-07-21'}

In [25]:
clss1.__dict__

{'id': None,
 'name': 'HC_2',
 'code': 'HC_2_2010-04-24_2010-10-09',
 'start_date': '2010-04-24',
 'end_date': '2010-10-09'}

In [26]:
cls_sql.save(clss0)

{'message': 'Cohort aleady exists'}


In [27]:
cls_sql.save(clss1)

{'message': 'Cohort aleady exists'}


In [28]:
cls_sql.get_by_name(cls_indx0['name'])
cls_sql.save(clss0)
cls_sql.get_by_id(1)
clss0.id
classes_ = cls_sql.get_all()

{'message': 'Cohort aleady exists'}


In [29]:
test_cls = classes['classes'].iloc[:2]

test_cls.to_records()

rec.array([(1, 'HC_1', 'HC_1_2009-02-03_2009-07-21', '2009-02-03', '2009-07-21'),
           (2, 'HC_2', 'HC_2_2010-04-24_2010-10-09', '2010-04-24', '2010-10-09')],
          dtype=[('index', '<i8'), ('name', 'O'), ('code', 'O'), ('start_date', 'O'), ('end_date', 'O')])

In [30]:
test_dict = test_cls.to_dict(orient='list')
test_dict.keys()

dict_keys(['name', 'code', 'start_date', 'end_date'])

In [31]:
for i in test_dict.keys():
    print(i)

name
code
start_date
end_date


In [32]:
classes.keys()

dict_keys(['classes'])

In [33]:
anon_test = classes['classes'].to_dict()
anon_test.keys()

dict_keys(['name', 'code', 'start_date', 'end_date'])

In [34]:
studentbyclass.keys()

dict_keys(['HC_1_students', 'HC_2_students', 'HC_3_students', 'HC_4_students', 'HC_5_students', 'HC_6_students', 'HC_7_students', 'HC_8_students', 'HC_9_students', 'HC_10_students', 'HC_11_students', 'HC_12_students'])

In [35]:
studentbyclass['HC_1_students']

,name,email,phone
1,Conor Goddard,conorgoddard@gmail.com,0151 496 0067
2,Joan Jenkins,joanjenkins@yahoo.com,0115 496 0699
3,Tracy Young,tracyyoung@hotmail.com,(0121)4960644
4,Dominic Ali,dominicali@yahoo.com,(0909) 879 0555
5,Clifford Webb,cliffordwebb@hotmail.com,0909 8790740
...,...,...,...
396,Nicole Brown,nicolebrown@hotmail.com,+44(0)306 9990975
397,Lorraine O'Connor,lorraineo'connor@hotmail.com,+44131 4960600
398,Frederick Hobbs,frederickhobbs@hotmail.com,+44909 879 0484
399,Amy Wells,amywells@hotmail.com,+44(0)1184960339


In [36]:
hc  = 'HC_14_students'

In [37]:
hc.rsplit('_', 1)[0]

'HC_14'

In [38]:
from flask_apps.models.Admissions import Admissions, AdmissionsSql
from flask_apps.models.People import People, PeopleSql
from flask_apps.models.Outcomes import Outcomes, OutcomesSql

people_sql = PeopleSql()
classes_sql = ClassesSql()
admissions_sql = AdmissionsSql()
outcomes_sql = OutcomesSql()

In [39]:

# admission_lst = []

for sheet in studentbyclass.keys():
    
    
    admissions_res = studentbyclass[sheet].to_dict(orient='records')
    
    for entry in admissions_res[:2]:
        class_name = sheet.rsplit('_',1)[0]
        email = entry.get('email')
        print(email)
        person_obj= people_sql.get_by_email(email)
        class_obj= classes_sql.get_by_name(class_name)
        
#         print(person_obj['first_name'])
        
    
#     for entry in admissions_res

conorgoddard@gmail.com
joanjenkins@yahoo.com
jaynebrown@gmail.com
traceyabbott@gmail.com
arthuredwards@gmail.com
lisacartwright@gmail.com
zoehopkins@hotmail.com
eileenhowell@gmail.com
leslieharris@hotmail.com
paulrowley@gmail.com
cameronwells@yahoo.com
martynbruce@yahoo.com
deniseroberts@hotmail.com
rhyshutchinsonZ@gmail.com
teresawong@yahoo.com
joanrobinson@yahoo.com
geraldhunt@gmail.com
joelleonard@hotmail.com
marcusbryan@yahoo.com
teresathornton@gmail.com
barrynorman@hotmail.com
leighroberts@gmail.com
colinsmith@yahoo.com
francescawalsh@yahoo.com


In [40]:
people_sql.get_by_email('conorgoddard@gmail.com')

[225, 'Conor', 'Goddard', '0151 496 0067', 'conorgoddard@gmail.com', 'M']

In [41]:
# admissions_res = studentbyclass['HC_1_students'].to_dict(orient='records')

# admissions_res[:5]

In [42]:
people_sql.get_by_email('jaynebrown@gmail.com')

[979, 'Jayne', 'Brown', '(0114) 4960032', 'jaynebrown@gmail.com', 'M']

In [43]:
from config import configs

# test = pd.read_sql_query(sql='SELECT * FROM classes',con=configs['dev'].DATABASE_URI)

with psycopg2.connect(configs['dev'].DATABASE_URI) as conn:
    test = pd.read_sql_query('SELECT * FROM classes',conn)

    

In [44]:
test

,id,class_name,code,start_date,end_date
0,1,HC_1,HC_1_2009-02-03_2009-07-21,2009-02-03,2009-07-21
1,2,HC_2,HC_2_2010-04-24_2010-10-09,2010-04-24,2010-10-09
2,3,HC_3,HC_3_2011-02-14_2011-08-01,2011-02-14,2011-08-01
3,4,HC_4,HC_4_2012-05-28_2012-11-12,2012-05-28,2012-11-12
4,5,HC_5,HC_5_2013-01-17_2013-07-04,2013-01-17,2013-07-04
5,6,HC_6,HC_6_2014-04-05_2014-09-20,2014-04-05,2014-09-20
6,7,HC_7,HC_7_2015-05-02_2015-10-17,2015-05-02,2015-10-17
7,8,HC_8,HC_8_2016-05-14_2016-10-29,2016-05-14,2016-10-29
8,9,HC_9,HC_9_2017-01-07_2017-06-24,2017-01-07,2017-06-24
9,10,HC_10,HC_10_2018-01-25_2018-07-12,2018-01-25,2018-07-12


In [45]:
from flask_apps.models.Applications import ApplicationsSql

In [46]:
applications_sql = ApplicationsSql()

application_df = applications_sql.get_df()

In [48]:
application_df

,id,person_id,class_id,date
0,1,1,1,2009-02-01
1,2,2,1,2009-01-22
2,3,3,1,2009-01-25
3,4,4,2,2010-03-20
4,5,5,1,2009-01-08
...,...,...,...,...
6491,6492,6492,12,2020-02-06
6492,6493,6493,12,2020-02-24
6493,6494,6494,12,2020-03-07
6494,6495,6495,12,2020-02-19


In [51]:
application_df.dtypes

id            int64
person_id     int64
class_id      int64
date         object
dtype: object

In [53]:
application_df['date'] = pd.to_datetime(application_df['date'])
application_df.dtypes

id                    int64
person_id             int64
class_id              int64
date         datetime64[ns]
dtype: object

In [54]:
application_df['year'] = application_df['date'].dt.year
application_df.head()

,id,person_id,class_id,date,year
0,1,1,1,2009-02-01,2009
1,2,2,1,2009-01-22,2009
2,3,3,1,2009-01-25,2009
3,4,4,2,2010-03-20,2010
4,5,5,1,2009-01-08,2009


In [50]:
application_df['class_id'].value_counts()

12    617
5     590
7     548
10    547
2     547
9     538
11    537
4     521
8     520
3     514
1     509
6     508
Name: class_id, dtype: int64

In [69]:
from flask_apps.models.Outcomes import OutcomesSql
from flask_apps.models.Classes import ClassesSql


outcomes_sql = OutcomesSql()
class_sql = ClassesSql()

outcomes_df = outcomes_sql.get_df()
classes_df = class_sql.get_df()

In [65]:
print(outcomes_df.shape)
outcomes_df.head()

(1065, 3)


,id,person_id,class_id
0,1,431,1
1,2,275,1
2,3,370,1
3,4,159,1
4,5,283,1


In [66]:
print(class_df.shape)
class_df.rename(columns={'id': 'class_id'}, inplace=True)
class_df.head()

(12, 5)


,class_id,class_name,code,start_date,end_date
0,1,HC_1,HC_1_2009-02-03_2009-07-21,2009-02-03,2009-07-21
1,2,HC_2,HC_2_2010-04-24_2010-10-09,2010-04-24,2010-10-09
2,3,HC_3,HC_3_2011-02-14_2011-08-01,2011-02-14,2011-08-01
3,4,HC_4,HC_4_2012-05-28_2012-11-12,2012-05-28,2012-11-12
4,5,HC_5,HC_5_2013-01-17_2013-07-04,2013-01-17,2013-07-04


In [68]:
class_cols = ['class_id', 'class_name', 'end_date']
out_class = pd.merge(outcomes_df, class_df[class_cols], how='inner', on='class_id')
print(out_class.shape)
out_class.head()

(1065, 5)


,id,person_id,class_id,class_name,end_date
0,1,431,1,HC_1,2009-07-21
1,2,275,1,HC_1,2009-07-21
2,3,370,1,HC_1,2009-07-21
3,4,159,1,HC_1,2009-07-21
4,5,283,1,HC_1,2009-07-21


In [71]:
class_applications = pd.merge(
    application_df, class_df.drop('code', axis=1), how='inner', on='class_id')
class_applications.head()

,id,person_id,class_id,date,year,class_name,start_date,end_date
0,1,1,1,2009-02-01,2009,HC_1,2009-02-03,2009-07-21
1,2,2,1,2009-01-22,2009,HC_1,2009-02-03,2009-07-21
2,3,3,1,2009-01-25,2009,HC_1,2009-02-03,2009-07-21
3,5,5,1,2009-01-08,2009,HC_1,2009-02-03,2009-07-21
4,6,6,1,2009-01-01,2009,HC_1,2009-02-03,2009-07-21


In [73]:
class_applications['quaters'] = class_applications['date'].dt.quarter

In [74]:
class_applications.head()

,id,person_id,class_id,date,year,class_name,start_date,end_date,quaters
0,1,1,1,2009-02-01,2009,HC_1,2009-02-03,2009-07-21,1
1,2,2,1,2009-01-22,2009,HC_1,2009-02-03,2009-07-21,1
2,3,3,1,2009-01-25,2009,HC_1,2009-02-03,2009-07-21,1
3,5,5,1,2009-01-08,2009,HC_1,2009-02-03,2009-07-21,1
4,6,6,1,2009-01-01,2009,HC_1,2009-02-03,2009-07-21,1
